In [1]:
import numpy as np
import helpers 

datapath = "./dataset/"


x_train, x_test, y_train, train_ids, test_ids = helpers.load_csv_data(datapath)

In [2]:
data_path = "./dataset/x_train.csv"
feature_names =  np.genfromtxt(data_path, max_rows=2, delimiter=',', names=True).dtype.names
feature_names = feature_names[1:] ## TODO , j'ai l'impression qu'il faut faire ça vue que x_train a pas ID
columns = np.asarray(feature_names[0:])
columns
x_test.shape

(109379, 321)

In [3]:


# TODO ATTENTION , dans chaque process ,     new_column = column.copy() 
# Python fait du passage par référence 
 

# TODO , all the variable in variables_for_process2 could be encode using one_hot_encoding_special,2,[7,9]

# 
"""
  
   on peux remplacer 

 for i in range(len(new_column)):
       if new_column[i] == 3:  
            new_column[i] = 0

    par ça , c'est plus court et aussi lisible 
    new_column[new_column==3]=0
 """

"\n  \n   on peux remplacer \n\n for i in range(len(new_column)):\n       if new_column[i] == 3:  \n            new_column[i] = 0\n\n    par ça , c'est plus court et aussi lisible \n    new_column[new_column==3]=0\n "

In [4]:

collumns_to_delete = []
D = x_train.shape[1]

for i in range(D-1):
    if len(np.where(np.isnan(x_train[:,i]))[0]) > x_train.shape[0]*0.60: #TODO Deleting collumns with >60% nan 
        collumns_to_delete.append(columns[i])



len(collumns_to_delete)

129

In [5]:
## Acess to collumns i : x_train[:,i]
x_new_train = np.copy(x_train) 
x_append_train = np.empty((x_train.shape[0],0))

x_new_test= np.copy(x_test) 
x_append_test = np.empty((x_test.shape[0],0))

def standardize(x):
    """Standardize the original data set."""
    mean_x = np.mean(x)
    x = x - mean_x
    std_x = np.std(x)
    x = x / std_x
    return x




def preprocessing_refactorisation(list_of_collumns_to_process,processing_function):
    for col in list_of_collumns_to_process:
        tab = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ]
        if (len(tab)>0):
            indice = tab[0]
            x_new_train[:,indice] = standardize(processing_function(x_train[:,indice]))
            x_new_test[:,indice] = standardize(processing_function(x_test[:,indice]))




# TODO CAREFULL , ONE hot Encoding increase the size of the size of the dataset

collumn_to_oneHotEncode = ["_STATE","IMONTH","IDAY","SEX","_DRDXAR1","_RACE_G1","_BMI5CAT","ACTIN11_","ACTIN21_"]
collumn_to_oneHotencode_special = [("MARITAL",6,[9]),("EDUCA",6,[9]),("RENTHOM1",3,[7,9]),("EMPLOY1",8,[9]),("INCOME2",8,[77,99]),("GENHLTH",5,[7,9]),
                                   ("HAREHAB1",2,[7,9]),("STREHAB1",2,[7,9]),("CVDASPRN",2,[7,9]),("ASPUNSAF",3,[7,9]),("RLIVPAIN",2,[7,9]),("RDUCHART",2,[7,9]),
                                   ("RDUCSTRK",2,[7,9]),("ARTHWGT",2,[7,9]),("ARTHEXER",2,[7,9]),("ARTHEDU",2,[7,9]),
                                   ("TETANUS",4,[7,9]),("HPVADVC2",3,[7,9]),("HPVADSHT",3,[77,99]),("SHINGLE2",2,[7,9]),
                                   ("SCNTLPAD",4,[7,9]),("SXORIENT",4,[7,9]),("TRNSGNDR",4,[7,9]),
                                   ("CASTHDX2",2,[7,9]),("CASTHNO2",2,[7,9]),("MISTMNT",2,[7,9]),("ADANXEV",2,[7,9]),("_CRACE1",7,[77,99]),
                                   ("_CPRACE",7,[77,99]),
                                   ("_RFHLTH",2,[9]),("_HCVU651",2,[9]),("_RFHYPE5",2,[9]),("_CHOLCHK",3,9),("_RFCHOL",2,[9]),("_LTASTH1",2,9),("_CASTHM1",2,[9]),
                                    ("_ASTHMS1",3,[9]),("_PRACE1",8,[77,99]),("_MRACE1",7,[77,99]),("_HISPANC",2,[9]),("_RACE",8,[9]),("_RACEG21",2,[9]),
                                    ("_RACEGR3",5,[9]),("_AGEG5YR",13,[14]),("_AGE65YR",2,[3]),("_RFBMI5",2,[9]),("_CHLDCNT",6,[9]),("_RFSMOK3",2,[9]),
                                    ("DRNKANY5",2,[7,9]),("_RFBING5",2,[9]),("_RFDRHV5",2,[9]),("_FRTLT1",2,[9]),("_VEGLT1",2,[9]),("_TOTINDA",2,[9]),
                                    ("_PAINDX1",2,[9]),("_PAREC1",4,[9])
                                   ]


collumn_to_oneHotencode_special_123 = [("PERSDOC2",3,[7,9]),("SMOKDAY2",3,[7,9])]
collumn_to_oneHotencode_special_pregnancy = [("BPHIGH4",4,[7,9]),("DIABETE3",4,[7,9]),("PREDIAB1",3,[7,9])]


collumn_to_oneHotencode_special= collumn_to_oneHotencode_special+collumn_to_oneHotencode_special_123+collumn_to_oneHotencode_special_pregnancy


print("hot encode special shape ",len(collumn_to_oneHotencode_special))
print("hot encode normal ",len(collumn_to_oneHotEncode))

#One hot encoded collumns must be deleted
collumns_to_delete_from_one_hot =["_STATE","IMONTH","IDAY",
                                  "SEX","MARITAL","EDUCA","RENTHOM1","EMPLOY1","INCOME2","GENHLTH",
                                  "PERSDOC2","SMOKDAY2",
                                  "BPHIGH4","DIABETE3",
                                  "HAREHAB1","STREHAB1","CVDASPRN","ASPUNSAF","RLIVPAIN","RDUCHART","RDUCSTRK","ARTHWGT","ARTHEXER","ARTHEDU",
                                  "TETANUS","HPVADVC2","HPVADSHT","SHINGLE2","SCNTLPAD","SXORIENT","TRNSGNDR","CASTHDX2","CASTHNO2","MISTMNT",
                                  "ADANXEV","_CRACE1","_CPRACE","_RFHLTH","_HCVU651","_RFHYPE5","_CHOLCHK","_RFCHOL","_MICHD","_LTASTH1","_CASTHM1",
                                  "_ASTHMS1","_DRDXAR1","_PRACE1","_MRACE1","_HISPANC","_RACE","_RACEG21","_RACEGR3","_RACE_G1",
                                  "_AGEG5YR","_AGE65YR","_BMI5CAT","_RFBMI5","_CHLDCNT","_RFSMOK3","DRNKANY5","_RFBING5","_RFDRHV5","_FRTLT1","_VEGLT1","_TOTINDA","ACTIN11_","ACTIN21_",
                                  "_PAINDX1","_PAREC1",
                                  ]


collumns_to_delete = collumns_to_delete+collumns_to_delete_from_one_hot



#One hot encoding of the collumn , Assumes that the collumn contains value between 1 and K
def one_hot_encoding(collumn):
    collumn[np.isnan(collumn)] = 0
    int_coll = collumn.astype(int)
    num_classes = max(int_coll)+1
    result = np.eye(num_classes)[int_coll]

    return result


# ONe hot encoding of the collumn , assumes that the collumns contains values between 1 and num_max , and that the value skip correspond to Nan / not answered ...

def one_hot_encoding_special(collumn,num_max,skip):
    collumn[np.isnan(collumn)] = 0
    collumn_as_int = collumn.astype(int)
    num_classes = num_max+1
    collumn_as_int[np.isin(collumn_as_int, skip)]=0
    result = np.eye(num_classes)[collumn_as_int]
    
    return result



for col in collumn_to_oneHotEncode :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1][0]
    encoded_train  = one_hot_encoding(x_train[:,indice])
    x_append_train = np.hstack((x_append_train,encoded_train))

    encoded_test = one_hot_encoding(x_test[:,indice])
    x_append_test = np.hstack((x_append_test,encoded_test))

    

for col,num_max,skip in collumn_to_oneHotencode_special:
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1][0]
    encoded_train  = one_hot_encoding_special(x_train[:,indice],num_max,skip)
    x_append_train = np.hstack((x_append_train,encoded_train))

    encoded_test  = one_hot_encoding_special(x_test[:,indice],num_max,skip)
    x_append_test = np.hstack((x_append_test,encoded_test))


x_append_train.shape,x_append_test.shape # Currently , shape 415

hot encode special shape  11
hot encode normal  9


((328135, 205), (109379, 205))

I think we should not consider IDATE, we should directly consider IMONTH, IDAY, IYEAR since it corresponds to the same thing, we should consider the columns to potentially drop

In [6]:


array_to_trop_list = ["DIS"]
array_to_drop_Useless = ["_PSU","SEQNO","CTELENUM","STATERES","CELLFON3","DISPCODE","PVTRESD1","CTELNUM1","CELLFON2",
                         "PVTRESD2","LANDLINE","HHADULT","NUMHHOL2","IMFVPLAC","WHRTST10","NUMADULT" , "NUMMEN" , "NUMWOMEN",
                         "RCSGENDR","RCSRLTN2","QSTVER","QSTLANG","EXACTOT1","EXACTOT2","MSCODE","_STSTR","_STRWT","_RAWRAKE","_WT2RAKE",
                         "_CHISPNC","_CLLCPWT","_DUALUSE","_DUALCOR",
                         "_AGE_G","HTIN4","WTKG3","HTM4",
                         
                         

                         "_MISFRTN","_MISVEGN","_FRTRESP","_VEGRESP","_FRT16","_VEG23","_FRUITEX","_VEGETEX",
                         "PAMISS1_","_LMTSCL1","_RFSEAT2","_RFSEAT3","_FLSHOT6","_PNEUMO2","_AIDTST3"

                         ,"CHOLCHK","FLSHTMY2","FEETCHK2","FEETCHK",
                         "LONGWTCH" #maybe use it later 
                         
                         
                         ]
array_to_drop_redundant = ["IDATE","FMONTH","IYEAR"] #IYEAR is useless , 99% in 2015
array_to_drop_too_many_missing =["LADULT","COLGHOUS","CADULT","CCLGHOUS","CSTATE","NUMPHON2", "PAINACT2","QLMENTL2","QLSTRES2","QLHLTH2",
                                 "CRGVREL1","CRGVLNG1","CRGVHRS1","CRGVPRB1","CRGVPERS", "CRGVHOUS","CRGVMST2","CRGVEXPT","HIVTSTD3","VIDFCLT2",
                                 "VIREDIF3","VIPRFVS2","VINOCRE2","VIEYEXM2","VIINSUR2","VICTRCT4","ASERVIST","ASDRVIST","ASRCHKUP","ASACTLIM","ASYMPTOM","ASNOSLEP",
                                 "ASTHMED3","ASINHALR"
                                 "PCPSADE1","PCDMDECN"
                                 ]
array_to_drop_or_to_oneHot =["EXRACT11","EXRACT21"]

collumns_to_delete = collumns_to_delete + array_to_trop_list+array_to_drop_Useless+array_to_drop_redundant+array_to_drop_too_many_missing+array_to_drop_or_to_oneHot

len(collumns_to_delete)

#CAREFULL , DO NOT RUN MULTIPLE TIME , 

298

Process for PHYSHLTH, MENTHLTH, POORHLTH 
The values corresponds to days, between 1-30 it is already good.
We're going to replace all 88 values by 0 because it corresponds to None which is 0 days, and we will assume that those who refused or did not answer have median values.

In [7]:
def process_HLTH(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if new_column[i] == 88: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 30]        
    median = np.nanmedian(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column


In [8]:
variables_for_process_HLTH = ["PHYSHLTH","MENTHLTH", "POORHLTH"]

preprocessing_refactorisation(variables_for_process_HLTH,process_HLTH)

Process for values that have as answer Yes or No
We're going to replace No by 0 and yes by 1, since yes is already by 1, we're going to change 2 by 0. For values 7 which corresponds to don't know or not sure, 9 for refused, and BLANK for missing values, we're going to take the median

We're going to use this method for variables : HLTHPLN1, MEDCOST, BPMEDS, BLOODCHO, TOLDHI2, CVDINFR4, CVDCRHD4, CVDSTRK3, ASTHMA3, ASTHNOW, CHCSCNCR, CHCOCNCR, CHCCOPD1, HAVARTH3, ADDEPEV2, CHCKIDNY, 

In [9]:
def process_2(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if new_column[i] == 2: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 1]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [10]:
variables_for_process2 = ["HLTHPLN1", "MEDCOST", "BPMEDS", "BLOODCHO", "TOLDHI2",
                          "CVDSTRK3", "ASTHMA3", "ASTHNOW",
                          "CHCSCNCR", "CHCOCNCR", "CHCCOPD1", "HAVARTH3", "ADDEPEV2",
                          "CHCKIDNY", "CPDEMO1", "VETERAN3", "INTERNET", "PREGNANT",
                         "QLACTLM2", "USEEQUIP", "BLIND", "DECIDE", "DIFFWALK", "DIFFDRES",
                          "DIFFALON", "SMOKE100","STOPSMK2", "EXERANY2",
                         "LMTJOIN3", "ARTHDIS2", "FLUSHOT6", "PNEUVAC3",
                         "HIVTST6", "PDIABTST", "INSULIN", "DIABEYE", "DIABEDU","CAREGIV1","VIGLUMA2","VIMACDG2","CIMEMLOS","CDDISCUS",
                         "WTCHSALT","DRADVISE","ASATTACK",
                         "HADMAM","HADPAP2","HPVTEST","HADHYST2","PROFEXAM","BLDSTOOL",
                         "HADSIGM3","HADSGCO1","PCPSAAD2","PCPSADI1","PCPSARE1","PSATEST1",
                         "_PA30021","_PASTRNG","_PASTAE1"
                         ]

preprocessing_refactorisation(variables_for_process2,process_2)

Process for CHECKUP1, we have to check what to do with never have been checked I gave 0 but I think we should instead do 5
see how we can do both CHOLCHK and CHECKUP1

In [11]:
def process_3(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if new_column[i] == 8: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 4]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [12]:
variables_for_process3 = ["CHECKUP1", "CHOLCHK"]

preprocessing_refactorisation(variables_for_process3,process_3)


See process for DIABAGE2, how old were you (ages between 1-97) ?

In [13]:
def process_4(column):
    new_column = column.copy()

    filtered_elements = [x for x in new_column if 1 <= x <= 97]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 98) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [14]:
# TODO DIABAGE2 has 384 204 missing values , do we keep it ? 

variables_for_process4 = ["DIABAGE2"]

preprocessing_refactorisation(variables_for_process4,process_4)

See process for CHILDREN, values between 1-87, None corresponds to 88 we will transform to 0

In [15]:
def process_5(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 88: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 87]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 98) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [16]:
variables_for_process5 = ["CHILDREN"]


preprocessing_refactorisation(variables_for_process5,process_5)

See process for WEIGHT2, For weight we will put everything in kilograms, so for values between 50-0999 which is in pound we will transform it it kg, and for values between 9000-9998 we will remove the first column of the value because it only corresponds to the fact that the value is in kg.

See process for HEIGHT3, we will transform everything in centimeters, from inches to centimers

In [17]:



def process_6_weight(column):
    new_column = column.copy()

    
    for i in range(len(new_column)):
        if new_column[i] >= 50 and new_column[i] <= 999: 
            new_column[i] = new_column[i] * 0.453592
        elif new_column[i] >= 9000 and new_column[i] <= 9998: 
            new_column[i] = new_column[i] - 9000

            
    filtered_elements = [x for x in new_column if 0 <= x <= 999]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7777) or (new_column[i] == 9999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

def process_6_height(column):
    new_column = column.copy()
    



    for i in range(len(new_column)):
        if new_column[i] >= 200 and new_column[i] <= 711: 

            feet = new_column[i]//100
            inch = (new_column[i]-feet*100)+feet*12
            new_column[i] = inch* 2.54
        elif new_column[i] >= 9000 and new_column[i] <= 9998: 
            new_column[i] = new_column[i] - 9000

       
            
    filtered_elements = [x for x in new_column if 0 <= x <= 999]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7777) or (new_column[i] == 9999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [18]:
preprocessing_refactorisation(["WEIGHT2"],process_6_weight)
preprocessing_refactorisation(["HEIGHT3"],process_6_height)

We do process for ALCDAY5 where :
- the value in the form 1_ _ = days per week, there are 4 week per month
- 2_ _ = days in the past 30 day
- 888 no drinks we will then consider 0
- for the values 777, 999, BLANK, we will consider the mean

The new value corresponds to the number of days per month you consumed at least one drink

Same thing works for EXEROFT1

In [19]:
def process_7(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if new_column[i] >= 101 and new_column[i] <= 199: 
            new_column[i] = (new_column[i] - 100) * 4
        elif new_column[i] >= 201 and new_column[i] <= 299: 
            new_column[i] = (new_column[i] - 200)
        elif (new_column[i] == 888): 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 99]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [20]:
variables_for_process7 = ["ALCDAY5", "EXEROFT1", "EXEROFT2", "STRENGTH"]



preprocessing_refactorisation(variables_for_process7,process_7)

Do process for Alcohol, especially for AVEDRNK2 and the other, we are considering if it is BLANK because it is not asked since the previous answer is no (888), which is the answer to the question have you had any alcoholic beverage. Since the answer to the variable name ALCDAY5 is no they will not ask the other questions. 

In [21]:
def process_8(column, column_ALCDAY5):
    new_column = column.copy()
    for i in range(len(new_column)):
        if (np.isnan(new_column[i]) and column_ALCDAY5[i] == 888): 
            new_column[i] = 0

            
    filtered_elements = [x for x in new_column if 0 <= x <= 76]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [22]:



variables_for_process8 = ["AVEDRNK2", "DRNK3GE5", "MAXDRNKS" ]
indice_ALCDAY5 = [i for i, item in enumerate(feature_names) if item.find("ALCDAY5") != -1 ][0]
for col in variables_for_process8 :
    indice = [i for i, item in enumerate(feature_names) if item.find(col) != -1 ][0]
    x_new_train[:,indice] = process_8(x_train[:,indice], x_train[:,indice_ALCDAY5])
    x_new_test[:,indice] = process_8(x_test[:,indice], x_test[:,indice_ALCDAY5])

Process for FRUITJU1, FRUIT1, FVBEANS, FVGREEN, FVORANG, VEGETAB1  . We are going to use  information about FRUITJUI1 to understand how the values in FRUIT1 works, indeed,
- the value in the form 1_ _ = times per day, we will consider there are 30 days per month
- 2_ _ = times per week, we will consider there are 4 weeks per month (maybe you want to consider more 4,..)
- 300 means less than one time per month, we will consider 0 
- 3_ _ = times per month
- 555 never we will then consider 0
- for the values 777, 999, BLANK we will consider the median

In [23]:
def process_9(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if new_column[i] >= 101 and new_column[i] <= 199: 
            new_column[i] = (new_column[i] - 100) * 30
        elif new_column[i] >= 201 and new_column[i] <= 299: 
            new_column[i] = (new_column[i] - 200) * 4
        elif (new_column[i] == 300) or (new_column[i] == 555): 
            new_column[i] = 0
        elif new_column[i] >= 301 and new_column[i] <= 399: 
            new_column[i] = (new_column[i] - 300)

            
    filtered_elements = [x for x in new_column if 0 <= x <= 99]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [24]:



variables_for_process9 = ["FRUITJU1", "FRUIT1", "FVBEANS",
                          "FVGREEN", "FVORANG", "VEGETAB1"]
preprocessing_refactorisation(variables_for_process9,process_9)

We do process for EXERHMM1, we have values in hours and minutes, we want it to be continious, we have _ _ _, the first value correspond to the number of hours, the two other values correspond to minutes, We will convert everything to minutes 
Also works for EXERHMM2

In [25]:
def process_10(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if ((new_column[i] >= 1 and new_column[i] <= 759) 
        or (new_column[i] >= 800 and new_column[i] <= 959)):             
            first_digit = new_column[i] // 100
            digits2_3 = new_column[i] % 100           
            new_column[i] = first_digit * 60 + digits2_3

            
    filtered_elements = [x for x in new_column if 0 <= x <= 600]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [26]:
variables_for_process10 = ["EXERHMM1", "EXERHMM2"]

preprocessing_refactorisation(variables_for_process10,process_10)

DO IMFVPLAC

We will process the values for BLDSUGAR, FEETCHK2
We want to have the times they checked something per year:
- the value in the form 1_ _ = times per day, we will consider there are 365 days in a year
- 2_ _ = times per week, we will consider there are 52 weeks per year (you may want to consider there are more (52,143))
- 3_ _ = times per month, we consider there are 12 months in a year
- 4_ _ = times per year
- 555 no feet, we will then consider 0
- 888 never, whichi means 0
- for the values 777, 999, BLANK we will consider the median


In [27]:
def process_11(column):
    
    new_column = column.copy()
    
    for i in range(len(new_column)):
        if new_column[i] >= 101 and new_column[i] <= 199: 
            new_column[i] = (new_column[i] - 100) * 365
        elif new_column[i] >= 201 and new_column[i] <= 299: 
            new_column[i] = (new_column[i] - 200) * 52
        elif new_column[i] >= 301 and new_column[i] <= 399: 
            new_column[i] = (new_column[i] - 300) * 12
        elif new_column[i] >= 401 and new_column[i] <= 499: 
            new_column[i] = new_column[i] - 400
        elif (new_column[i] == 300) or (new_column[i] == 555): 
            new_column[i] = 0


            
    filtered_elements = [x for x in new_column if 0 <= x <= 99]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 777) or (new_column[i] == 999) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [28]:
variables_for_process11 = ["BLDSUGAR", "FEETCHK2"]
preprocessing_refactorisation(variables_for_process11,process_11)


Process for DOCTDIAB, CHKHEMO3, FEETCHK
We want all the values to be between 0-76 that corresponds to how many times they did something
- 88 which corresponds to None/ 0
- 98 never heard of means they have done it 0 times
- for the values 77-99-BLANK we will again consider the median

In [29]:
def process_12(column):
    new_column = column.copy()
    for i in range(len(new_column)):
        if (new_column[i] == 88 or new_column[i] == 98 ): 
            new_column[i] = 0

    filtered_elements = [x for x in new_column if 0 <= x <= 76]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [30]:
variables_for_process12 = ["DOCTDIAB","CHKHEMO3","FEETCHK"]
preprocessing_refactorisation(variables_for_process12,process_12)


/Users/malaklahlou/Library/Python/3.10/lib/python/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/malaklahlou/Library/Python/3.10/lib/python/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [31]:


def process_13(column):
    new_column = column.copy()
    # Never is mapped to 0 , some days mapped to 1 and every day mapped to 2 
    new_column[new_column==3]=0
    new_column[new_column==2]=1
    new_column[new_column==1]=2
            
    filtered_elements = [x for x in new_column if 0 <= x <= 2]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])): # DON'T KNOW / REFUSED 
            new_column[i] = median
            
    return new_column




In [32]:
variables_for_process13 = ["USENOW3","ARTHSOCL","SMOKDAY2","_PA150R2","_PA300R2","_LMTACT1","_LMTWRK1"]
preprocessing_refactorisation(variables_for_process13,process_13)

In [33]:
def process_14(column):
    new_column = column.copy()
            
    filtered_elements = [x for x in new_column if 0 <= x <= 10]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])): # DON'T KNOW / REFUSED 
            new_column[i] = median
            
    return new_column

In [34]:
variables_for_process14 = ["JOINPAIN"]
preprocessing_refactorisation(variables_for_process14,process_14)

In [35]:
def process_15(column):
    new_column = column.copy()
    new_column[new_column==8]=5

            
    filtered_elements = [x for x in new_column if 0 <= x <= 5]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])): # DON'T KNOW / REFUSED 
            new_column[i] = median
            
    return new_column

In [36]:
variables_for_process15 = ["EYEEXAM","ARTTODAY","SCNTMNY1","SCNTMEL1","SCNTPAID","_EDUCAG","_INCOMG","_SMOKER3","_PACAT1"]
preprocessing_refactorisation(variables_for_process15,process_15)

In [37]:
def process_16(column):
    new_column = column.copy()

    new_column[new_column==8]=1

            
    filtered_elements = [x for x in new_column if 0 <= x <= 6]
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])): # DON'T KNOW / REFUSED 
            new_column[i] = median
            
    return new_column

In [38]:
variables_for_process16 = ["SEATBELT","CDHOUSE","CDASSIST","CDHELP","CDSOCIAL","HOWLONG",
                           "LASTPAP2","HPLSTTST","LENGEXAM","LSTBLDS3","LASTSIG3","PSATIME","PCPSARS1",
                           "EMTSUPRT","LSATISFY"]
preprocessing_refactorisation(variables_for_process16,process_16)

In [39]:
def process_17(column):
    new_column = column.copy()



            
    filtered_elements = [x for x in new_column if 0 <= x <= 8]
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77) or (new_column[i] == 99) or np.isnan(new_column[i])): # DON'T KNOW / REFUSED 
            new_column[i] = median
            
    return new_column

In [40]:
variables_for_process17 = ["LASTSMK2"]
preprocessing_refactorisation(variables_for_process17,process_17)

In [41]:
def process_18(column):
    new_column = column.copy()

    filtered_elements = [x for x in new_column if 11 <= x <= 96]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 98) or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [42]:
variables_for_process18 = ["ASTHMAGE"]
preprocessing_refactorisation(variables_for_process18,process_18)

In [43]:
def process_19(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 98: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 96]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 97)  or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [44]:
variables_for_process19 = ["SCNTWRK1","SCNTLWK1"]
preprocessing_refactorisation(variables_for_process19,process_19)

In [45]:
def process_20(column):
    new_column = column
    for i in range(len(new_column)):
        if new_column[i] == 88: 
            new_column[i] = 0
            
    filtered_elements = [x for x in new_column if 0 <= x <= 14]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 77)  or (new_column[i] == 99) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [46]:
variables_for_process20 = ["ADPLEASR","ADDOWN","ADSLEEP","ADENERGY","ADEAT1","ADFAIL","ADTHINK","ADMOVE"]
preprocessing_refactorisation(variables_for_process20,process_20)

In [47]:
def process_21(column):
    new_column = column
 
    filtered_elements = [x for x in new_column if 0 <= x <= 99999]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ( np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [48]:
variables_for_process21 = ["GRENDAY_","ORNGDAY_","VEGEDA1_","FTJUDA1_","FRUTDA1_","BEANDAY_","_FRUTSUM","_VEGESUM","PADUR1_","PADUR2_",
                           "_MINAC11","_MINAC21","PAMIN11_","PAMIN21_","PA1MIN_","PAVIG11_","PAVIG21_","PA1VIGM_"]
preprocessing_refactorisation(variables_for_process21,process_21)

In [49]:
def process_22(column):
    new_column = column
 
    filtered_elements = [x for x in new_column if 0 <= x <= 128]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ( np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [50]:
variables_for_process22 = ["METVL11_","METVL21_"]
preprocessing_refactorisation(variables_for_process22,process_22)

In [51]:
def process_23(column):
    new_column = column
 
    filtered_elements = [x for x in new_column if 0 <= x <= 98999]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 99900) or (new_column[i]==99000) or  np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [52]:
variables_for_process23 = ["MAXVO2_","FC60_","PAFREQ1_","PAFREQ2_","STRFREQ_"]
preprocessing_refactorisation(variables_for_process23,process_23)

In [53]:
def process_24(column):
    new_column = column
 
    filtered_elements = [x for x in new_column if 0 <= x <= 9999]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if (np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [54]:
variables_for_process24 = ["_BMI5"]
preprocessing_refactorisation(variables_for_process24,process_24)

In [59]:
def process_25(column):
    
    """
    Process for BPHIGH4, 
    We consider 0 : don't have high blood pressure 
    we consider 1 : you have high blood pressure
    we consider 2 : to have borderline high which is even more than high blood pressure 
    - If told yes only during pregnancy that means no since it's normal 
   Same process but with DIABETE3, PREDIAB1, which corresponds to diabetes
   
   """
    new_column = column.copy()
    new_column[new_column==2]=0
    new_column[new_column==3]=0
    new_column[new_column==4]=2
            
    filtered_elements = [x for x in new_column if 0 <= x <= 2]        
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7)  or (new_column[i] == 9) or np.isnan(new_column[i])):
            new_column[i] = median
            
    return new_column

In [60]:
variables_for_process25 = ["BPHIGH4", "DIABETE3", "PREDIAB1"]
preprocessing_refactorisation(variables_for_process25,process_25)

In [61]:
def process_26(column):
    
    """
    Process for PERSDOC2, 
    We consider 0 : don't have a care provider, so 3 becomes 0 
    we consider 1 : you one
    we consider 2 : you have two
   """
    
    new_column = column.copy()

    new_column[new_column==3]=0

    filtered_elements = [x for x in new_column if 0 <= x <= 6]
    median = np.median(filtered_elements)
        
    for i in range(len(new_column)):
        if ((new_column[i] == 7) or (new_column[i] == 9) or np.isnan(new_column[i])): # DON'T KNOW / REFUSED 
            new_column[i] = median
            
    return new_column

In [62]:
variables_for_process26 = ["PERSDOC2"]
preprocessing_refactorisation(variables_for_process26,process_26)

collumn_to_oneHotencode_special = [("MARITAL",6,[9]),("RENTHOM1",3,[7,9]),("EMPLOY1",8,[9]), 
                                       ("_HCVU651",2,[9]),("_RFHYPE5",2,[9]),("_CHOLCHK",3,9),("_RFCHOL",2,[9]),("_LTASTH1",2,9),("_CASTHM1",2,[9]),
                                       ("_ASTHMS1",3,[9]),("_PRACE1",8,[77,99]),("_MRACE1",7,[77,99]),("_HISPANC",2,[9]),("_RACE",8,[9]),("_RACEG21",2,[9]),
                                       ("_RACEGR3",5,[9]),("_AGEG5YR",13,[14]),("_AGE65YR",2,[3]),("_RFBMI5",2,[9]),("_CHLDCNT",6,[9]),("_RFSMOK3",2,[9]),
                                       ("DRNKANY5",2,[7,9]),("_RFBING5",2,[9]),("_RFDRHV5",2,[9]),("_FRTLT1",2,[9]),("_VEGLT1",2,[9]),("_TOTINDA",2,[9]),
                                       ("_PAINDX1",2,[9]),("_PAREC1",4,[9]),("PERSDOC2",3,[7,9]),("SMOKDAY2",3,[7,9]),("BPHIGH4",4,[7,9]),("DIABETE3",4,[7,9]),
                                       ("PREDIAB1",3,[7,9]),("IMFVPLAC",11,[77,99]),("WHRTST10",9,[77,99]),("_DUALUSE",2,[9]),("_AGE_G",6,[]),("_LMTSCL1",4,[9]),
                                       ("_RFSEAT3",2,[9]),("_FLSHOT6",2,[9]),("_PNEUMO2",2,[9])]


In [63]:
#TODO THIS LINE OF CODE MUST BE AT THE END OF THE NOTEBOOK 








collumns_not_find = {"DIS","EXACTOT1","QLMENTL2","EXACTOT2","ASINHALRPCPSADE1","PAINACT2","_MICHD","QLSTRES2","QLHLTH2"}
collumns_to_delete = list(set(collumns_to_delete)-collumns_not_find) #remove duplicates


indice_to_delete = [feature_names.index(item) for item in collumns_to_delete]




indice_to_delete.sort()

x_new_del_train = np.delete(x_new_train,indice_to_delete,1)
x_new_del_test = np.delete(x_new_test,indice_to_delete,1)





In [64]:
x_new_del_train = np.hstack((x_new_del_train,x_append_train))
x_new_del_test = np.hstack((x_new_del_test,x_append_test))

In [65]:
x_new_del_train.shape,x_new_del_test.shape

((328135, 298), (109379, 298))

We can remove columns with 0 variance 

In [66]:
def remove_zero_variance_columns(x) :
    
    variances = np.var(x, axis=0)
    zero_variance_columns = np.where(variances == 0)[0]
    x = np.delete(x, zero_variance_columns, axis=1)
    
    return x, zero_variance_columns
    

In [67]:
x_new_del_train.shape

(328135, 298)

In [68]:
x_new_del_train, to_delete = remove_zero_variance_columns(x_new_del_train)
x_new_del_test = np.delete(x_new_del_test, to_delete, axis=1 )

In [69]:
x_new_del_train.shape

(328135, 275)

Removes correlated columns, leaving only one column for each originally correlated sets of columns.

In [70]:
def remove_correlated_columns(x , threshold: float = 0.9) :
    """
    Removes correlated columns, leaving only one column for each originally correlated sets of columns.

    Arguments
    ---------
    x: np.ndarray
        The matrix to be cleaned.

    threshold: float
        The maximum percentage of correlation allowed (default = 0.9).

    Returns
    -------
    clean_x: np.ndarray
        The cleaned matrix.

    to_keep: nd.array
        Boolean array of the kept columns.
    """

    assert 0 <= threshold <= 1
    _, to_remove = np.where(np.triu(np.corrcoef(x.T), 1) > threshold)
    to_remove = list(set(to_remove))
    return np.delete(x, to_remove, axis=1), to_remove


In [71]:
x_new_del_train, to_remove = remove_correlated_columns(x_new_del_train)

In [72]:
x_new_del_train.shape , x_new_del_test.shape

((328135, 255), (109379, 275))

In [73]:
x_new_del_test = np.delete(x_new_del_test, to_remove, axis=1)

In [74]:
x_new_del_train.shape,x_new_del_test.shape

((328135, 255), (109379, 255))

In [75]:
print(np.count_nonzero(np.isnan(x_new_del_train)))
print(np.count_nonzero(np.isnan(x_new_del_test)))

0
0


In [76]:
def calculate_parameters(y_test, y_pred):
    """Calculates the parameters for the metrics
    
    Args:
        y_test: test labels
        y_pred: predicted labels
        
    Returns:
        tp: true positive
        tn: true negative
        fp: false positive
        fn: false negative
    """
    #print(y_test.shape, y_pred.shape)
    tp = sum((yt == 1) & (yp == 1) for yt, yp in zip(y_test, y_pred))
    tn = sum((yt == -1) & (yp == -1) for yt, yp in zip(y_test, y_pred))
    fp = sum((yt == -1) & (yp == 1) for yt, yp in zip(y_test, y_pred))
    fn = sum((yt == 1) & (yp == -1) for yt, yp in zip(y_test, y_pred))
    return tp, tn, fp, fn


def precision(tp, fp):
    """Calculates the precision score, which is the proportion of positive identifications that are actually correct
    
    Args: 
        tp: true positive
        fp: false positive
        
    Returns:
        precision score
    
    """
    if(tp + fp == 0):
        return 0.0
    return tp / (tp + fp)

def recall(tp, fn):
    """Calculates the recall score, which is the proportion of positives correctly identified. 
    It is different from precision in that it does not take into account the false positives.
    
    Args:
        tp: true positive
        fn: false negative
        
    Returns:
        recall score
    """
    if(tp + fn == 0):
        return 0.0
    return tp / (tp + fn)

def f1_score(tp, fp, fn):
    """Calculates the f1 score, which is the harmonic mean of precision and recall
    
    Args:
        tp: true positive
        fp: false positive
        fn: false negative
        
    Returns:
        f1 score
    """
    denom = precision(tp, fp) + recall(tp, fn)
    if(denom == 0):
        return 0.0
    return 2 * precision(tp, fp) * recall(tp, fn) / denom

def accuracy(tp, tn, fp, fn):
    """Calculates the accuracy score, which is the proportion of correct predictions
    
    Args:
        tp: true positive
        tn: true negative
        fp: false positive
        fn: false negative
        
    Returns:
        accuracy score
    """
    denom = (tp + tn + fp + fn)
    if(denom == 0):
        return 0.0
    return (tp + tn) / denom

In [77]:

import implementations
import random

def cross_validation_log(x, y, k_folds, hyperparameters):

    # Lists to store performance metrics for each fold and lambda
    f1_scores = []
    accuracy_scores = []

    # Split the data into k_fold parts
    fold_size = len(x) // k_folds
    
    # Dictionary to store results for each set of hyperparameters
    results = {}
    
    for lambda_ in hyperparameters:

        for k in range(k_folds):
            # Split the data into training and validation sets
            start = k * fold_size
            end = (k + 1) * fold_size
            x_valid = x[start:end]
            y_valid = y[start:end]
            x_train_fold = np.concatenate((x[:start], x[end:]))
            y_train_fold = np.concatenate((y[:start], y[end:]))
            
            # Train the model using logistic or regularized logistic regression
            w, loss = implementations.ridge_regression(
                y_train_fold, 
                x_train_fold, 
                lambda_, 
            )
               
            y_pred = x_valid.dot(w)
            y_pred=(y_pred>0)*2-1

            #print("y_pred is:" ,    y_pred)
            # # Make predictions on the validation set
            # y_pred = np.sign(x_valid.dot(w)) # this function returns -1 or 1
            
            # Calculate performance metrics
            tp, tn, fp, fn = calculate_parameters(y_valid, y_pred)
            f1_scores.append(f1_score(tp, fp, fn))
            accuracy_scores.append(accuracy(tp, tn, fp, fn))
            
        # Calculate the mean performance metrics for each set of hyperparameters
        mean_f1 = np.mean(f1_scores)
        mean_accuracy = np.mean(accuracy_scores)
        
        # Store the results in the dictionary
        results[str(lambda_)] = {'F1 score': mean_f1, 'Accuracy': mean_accuracy, 'param w ':w}
        
    return results


def generate_random_hyperparameters(num_samples):
    """Generates a list of random hyperparameters
    
    Args:
        num_samples: number of sets of hyperparameters to generate
        num_features: number of features in the dataset
        
    Returns:
        hyperparameters_list: list of hyperparameters
    """



    return np.linspace(1e-16, 0.001, num_samples)

In [78]:

num_samples = 5

num_features = x_new_del_train.shape[1]

random_hyperparameters = generate_random_hyperparameters(num_samples)


k_folds = 5

results = cross_validation_log(x_new_del_train, y_train, k_folds, random_hyperparameters)
print(results)

{'1e-16': {'F1 score': 0.0911480624309216, 'Accuracy': 0.9138494826824326, 'param w ': array([-1.74389774e-04, -4.27210396e-03,  3.11942969e-05,  3.70205457e-03,
        5.23796478e-03, -7.05804011e-05,  2.36410695e-02, -7.57600060e-04,
        4.02246227e-02,  6.56989861e-02,  1.24127298e-04,  8.02843837e-03,
       -1.82276851e-03,  4.06189576e-02,  6.76776047e-02,  2.00041432e-03,
        2.78136645e-02,  3.44500128e-03,  1.96367549e-02,  7.49195147e-04,
       -7.29721397e-03, -4.96152595e-03, -4.50317095e-03,  4.28405875e-03,
        1.25945046e-02,  9.90369704e-03, -2.00865831e-03,  1.42970781e-02,
       -3.06132707e-03, -7.59049786e-05,  7.50035383e-02, -2.65490099e-03,
       -7.15571136e-03, -1.02307296e-03, -2.83292200e-05, -1.19229589e-03,
       -2.68652898e-03, -9.36565546e-03,  1.45441890e-03,  1.40918699e-03,
       -7.05639236e-04,  3.19176888e-03,  2.31645064e-03, -1.13423196e-03,
       -4.84795480e-03, -5.56022471e-04, -1.06845158e-03,  2.20183560e-03,
        5.456

In [79]:
for a in results:
    print(results[a]["F1 score"])
    print(a)

0.0911480624309216
1e-16
0.09114040471118205
0.00025000000000007503
0.09110013178275676
0.00050000000000005
0.09109516251675967
0.000750000000000025
0.09103247578965398
0.001


In [80]:
# Best model found so far : 

# ridge_regression with lambda = 1e-16  : f1 score =13831

In [81]:
import implementations
import importlib
importlib.reload(implementations)
from implementations import *




initial_w = np.zeros(x_new_del_train.shape[1]) 

## Todo better initialization ? maybe normal distributed




MaxIter = 100000
gamma = 0.01

w,loss = ridge_regression(y_train, x_new_del_train,0.01) ## GAVE 0.063 F1 SCORE , maybe use lambda really small to approximate a least square 

#w,loss = implementations.logistic_regression(y_train, x_new_del_train, initial_w, MaxIter, gamma)


w.shape,x_new_del_test.shape





y_pred4 = x_new_del_test.dot(w)
y_pred4=(y_pred4>0)*2-1
y_pred4

helpers.create_csv_submission(test_ids, y_pred4, "Submission4.csv")

In [412]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression().fit(x_new_del_train,y_train)
y_pred3 = clf.predict(x_new_del_test)
y_pred3 = (y_pred3>0)*2-1
helpers.create_csv_submission(test_ids, y_pred3, "Submission3.csv") # gave 0.132 F1 score , Problem , we're not allowed to use it for the final submission